### Topological sort

#### Topological sort
Given vertices and edges, find the topological sort order

In [1]:
# 

from collections import deque


def topological_sort(vertices, edges):
    sortedOrder = []
    if vertices <= 0:
        return sortedOrder

    # a. Initialize the graph
    inDegree = {i: 0 for i in range(vertices)}  # count of incoming edges
    graph = {i: [] for i in range(vertices)}  # adjacency list graph

    # b. Build the graph
    for edge in edges:
        parent, child = edge[0], edge[1]
        graph[parent].append(child)  # put the child into it's parent's list
        inDegree[child] += 1  # increment child's inDegree

    # c. Find all sources i.e., all vertices with 0 in-degrees
    sources = deque()
    for key in inDegree:
        if inDegree[key] == 0:
            sources.append(key)

    # d. For each source, add it to the sortedOrder and subtract '1' from all of its 
    # children's in-degrees if a child's in-degree becomes zero, add it to sources queue
    while sources:
        vertex = sources.popleft()
        sortedOrder.append(vertex)
        for child in graph[vertex]:  # get the node's children to decrement their in-degrees
            inDegree[child] -= 1
            if inDegree[child] == 0:
                sources.append(child)

    # topological sort is not possible as the graph has a cycle
    if len(sortedOrder) != vertices:
        return []

    return sortedOrder

In [2]:
print (topological_sort(4, [[3, 2], [3, 0], [2, 0], [2, 1]]))

[3, 2, 0, 1]


#### Task scheduling
given a list of requirements in the form [0, 1][1, 2] => 0 needs to be done before 1 & 1 needs to be done before 2
return true if a schedule is possible (or in other words a topo sort exists)
(If asked to return the order, return the sortedOrder)

In [4]:
from collections import deque


def is_scheduling_possible(tasks, prerequisites):
    sortedOrder = []
    if tasks <= 0:
        return False

    # a. Initialize the graph
    inDegree = {i: 0 for i in range(tasks)}  # count of incoming edges
    graph = {i: [] for i in range(tasks)}  # adjacency list graph

    # b. Build the graph
    for prerequisite in prerequisites:
        parent, child = prerequisite[0], prerequisite[1]
        graph[parent].append(child)  # put the child into it's parent's list
        inDegree[child] += 1  # increment child's inDegree

    # c. Find all sources i.e., all vertices with 0 in-degrees
    sources = deque()
    for key in inDegree:
        if inDegree[key] == 0:
            sources.append(key)

    # d. For each source, add it to the sortedOrder and subtract one from all of its 
    # children's in-degrees if a child's in-degree becomes zero, add it to sources queue
    while sources:
        vertex = sources.popleft()
        sortedOrder.append(vertex)
        for child in graph[vertex]:  # get the node's children to decrement their in-degrees
            inDegree[child] -= 1
            if inDegree[child] == 0:
                sources.append(child)

    # if sortedOrder doesn't contain all tasks, there is a cyclic dependency between 
    # tasks, therefore, we will not be able to schedule all tasks
    return len(sortedOrder) == tasks

In [5]:
print (is_scheduling_possible(3, [[0, 1], [1, 2], [2, 0]]))

False


#### Return all possible schedules

In [7]:
# need understanding of backtracking pattern for this (think of N-queens or sudoku)

from collections import deque


def print_orders(tasks, prerequisites):
    sortedOrder = []
    if tasks <= 0:
        return False

    # a. Initialize the graph
    inDegree = {i: 0 for i in range(tasks)}  # count of incoming edges
    graph = {i: [] for i in range(tasks)}  # adjacency list graph

    # b. Build the graph
    for prerequisite in prerequisites:
        parent, child = prerequisite[0], prerequisite[1]
        graph[parent].append(child)  # put the child into it's parent's list
        inDegree[child] += 1  # increment child's inDegree

    # c. Find all sources i.e., all vertices with 0 in-degrees
    sources = deque()
    for key in inDegree:
        if inDegree[key] == 0:
            sources.append(key)

    print_all_topological_sorts(graph, inDegree, sources, sortedOrder)


def print_all_topological_sorts(graph, inDegree, sources, sortedOrder):
    if sources:
        for vertex in sources:
            sortedOrder.append(vertex)
            sourcesForNextCall = deque(sources)  # make a copy of sources
            # only remove the current source, all other sources should remain in the queue for
            # the next call
            sourcesForNextCall.remove(vertex)
            # get the node's children to decrement their in-degrees
            for child in graph[vertex]:
                inDegree[child] -= 1
                if inDegree[child] == 0:
                    sourcesForNextCall.append(child)

            # recursive call to print other orderings from the remaining (and new) sources
            print_all_topological_sorts(graph, inDegree, sourcesForNextCall, sortedOrder)

            # backtrack, remove the vertex from the sorted order and put all of its children 
            # back to consider the next source instead of the current vertex
            sortedOrder.remove(vertex)
            for child in graph[vertex]:
                inDegree[child] += 1

    # if sortedOrder doesn't contain all tasks, either we've a cyclic dependency between 
    # tasks, or we have not processed all the tasks in this recursive call
    if len(sortedOrder) == len(inDegree):
        print(sortedOrder)

In [9]:
print_orders(3, [[0, 1], [1, 2]])

print_orders(4, [[3, 2], [3, 0], [2, 0], [2, 1]])

[0, 1, 2]
[3, 2, 0, 1]
[3, 2, 1, 0]


#### Alien dictionary
Need to use lexicographic ordering from the input
Solution is to first build the graph from the input, then find the topo sort

In [12]:
from collections import deque


def find_order(words):
    if len(words) == 0:
        return ""

    # a. Initialize the graph
    inDegree = {}  # count of incoming edges
    graph = {}  # adjacency list graph
    for word in words:
        for character in word:
            inDegree[character] = 0
            graph[character] = []

    # b. Build the graph
    for i in range(0, len(words)-1):
        # find ordering of characters from adjacent words
        w1, w2 = words[i], words[i + 1]
        for j in range(0, min(len(w1), len(w2))):
            parent, child = w1[j], w2[j]
            if parent != child:  # if the two characters are different
                # put the child into it's parent's list
                graph[parent].append(child)
                inDegree[child] += 1  # increment child's inDegree
                break   # only the first different character between the two words will help us 
                        # find the order

    # c. Find all sources i.e., all vertices with 0 in-degrees
    sources = deque()
    for key in inDegree:
        if inDegree[key] == 0:
            sources.append(key)

    # d. For each source, add it to the sortedOrder and subtract one from all of its 
    # children's in-degrees if a child's in-degree becomes zero, add it to sources queue
    sortedOrder = []
    while sources:
        vertex = sources.popleft()
        sortedOrder.append(vertex)
        for child in graph[vertex]:  # get the node's children to decrement their in-degrees
            inDegree[child] -= 1
            if inDegree[child] == 0:
                sources.append(child)

    # if sortedOrder doesn't contain all characters, there is a cyclic dependency between 
    # characters, therefore, we'll not be able to find the correct ordering of characters
    if len(sortedOrder) != len(inDegree):
        return ""

    return ''.join(sortedOrder)

In [13]:
print (find_order(["ba", "bc", "ac", "cab"]))

bac
